Code for the analysis of a small toy network for chapter 4 of my final year project


In [3]:
##Imports 
import numpy as np
import pandas as pd
import networkx as nx

In [4]:
print('hello world')

hello world


In [27]:
#make adjacency matrix#

A = np.zeros((6,6))
print(A)
links = [[0,4,3],[0,2,1],[1,2,1],[1,5,3],[2,3,2],[3,4,1],[3,5,1]]
for i in links:
    A[i[0]][i[1]] = i[2]
    A[i[1]][i[0]] = i[2]    

print(A)





[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[0. 0. 1. 0. 3. 0.]
 [0. 0. 1. 0. 0. 3.]
 [1. 1. 0. 2. 0. 0.]
 [0. 0. 2. 0. 1. 1.]
 [3. 0. 0. 1. 0. 0.]
 [0. 3. 0. 1. 0. 0.]]


In [41]:
#demands#

d15 = 1
d16 = 0
d25 = 0
d26 = 1
d = [d15,d16,d25,d26]

In [42]:
#Create graph#

g = nx.convert_matrix.from_numpy_matrix(A)

print(g.edges(data=True))

[(0, 2, {'weight': 1.0}), (0, 4, {'weight': 3.0}), (1, 2, {'weight': 1.0}), (1, 5, {'weight': 3.0}), (2, 3, {'weight': 2.0}), (3, 4, {'weight': 1.0}), (3, 5, {'weight': 1.0})]


In [43]:
#Add cyclepath tag to edges#
for u,v,d in g.edges(data=True):
    d['cycle'] = False
    
print(g.edges(data=True))

[(0, 2, {'weight': 1.0, 'cycle': False}), (0, 4, {'weight': 3.0, 'cycle': False}), (1, 2, {'weight': 1.0, 'cycle': False}), (1, 5, {'weight': 3.0, 'cycle': False}), (2, 3, {'weight': 2.0, 'cycle': False}), (3, 4, {'weight': 1.0, 'cycle': False}), (3, 5, {'weight': 1.0, 'cycle': False})]


In [44]:
def designate_infrastructure(g,cycleEdges):
    #edges is a list of the indices of edges for which cycling infrastructure will be applied#
    for idx in cycleEdges:
        g.edges[(idx[0],idx[-1])]['cycle'] = True
    
    return g
        



In [45]:
print(g.edges(data=True))
cycleEdges = [[2,3],[3,4]]
gCycleDesignated = designate_infrastructure(g,cycleEdges)
print(gCycleDesignated.edges(data=True))

[(0, 2, {'weight': 1.0, 'cycle': False}), (0, 4, {'weight': 3.0, 'cycle': False}), (1, 2, {'weight': 1.0, 'cycle': False}), (1, 5, {'weight': 3.0, 'cycle': False}), (2, 3, {'weight': 2.0, 'cycle': False}), (3, 4, {'weight': 1.0, 'cycle': False}), (3, 5, {'weight': 1.0, 'cycle': False})]
[(0, 2, {'weight': 1.0, 'cycle': False}), (0, 4, {'weight': 3.0, 'cycle': False}), (1, 2, {'weight': 1.0, 'cycle': False}), (1, 5, {'weight': 3.0, 'cycle': False}), (2, 3, {'weight': 2.0, 'cycle': True}), (3, 4, {'weight': 1.0, 'cycle': True}), (3, 5, {'weight': 1.0, 'cycle': False})]


In [48]:
def adjust_weights(g,omega):
    #adjust the weights of edges that are not cycle paths#
    for u,v,d in g.edges(data=True):
        if d['cycle'] == False:
            d['weight'] = d['weight'] * omega
            
    return g



In [49]:
print(g.edges(data = True))

print(gCycleDesignated.edges(data = True))

gAdjusted = adjust_weights(gCycleDesignated,2)

print(gAdjusted.edges(data = True))


[(0, 2, {'weight': 1.0, 'cycle': False}), (0, 4, {'weight': 3.0, 'cycle': False}), (1, 2, {'weight': 1.0, 'cycle': False}), (1, 5, {'weight': 3.0, 'cycle': False}), (2, 3, {'weight': 4.0, 'cycle': True}), (3, 4, {'weight': 2.0, 'cycle': True}), (3, 5, {'weight': 1.0, 'cycle': False})]
[(0, 2, {'weight': 1.0, 'cycle': False}), (0, 4, {'weight': 3.0, 'cycle': False}), (1, 2, {'weight': 1.0, 'cycle': False}), (1, 5, {'weight': 3.0, 'cycle': False}), (2, 3, {'weight': 4.0, 'cycle': True}), (3, 4, {'weight': 2.0, 'cycle': True}), (3, 5, {'weight': 1.0, 'cycle': False})]
[(0, 2, {'weight': 2.0, 'cycle': False}), (0, 4, {'weight': 6.0, 'cycle': False}), (1, 2, {'weight': 2.0, 'cycle': False}), (1, 5, {'weight': 6.0, 'cycle': False}), (2, 3, {'weight': 4.0, 'cycle': True}), (3, 4, {'weight': 2.0, 'cycle': True}), (3, 5, {'weight': 2.0, 'cycle': False})]
